In [ ]:
import os
import sys
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ResourceClosedError, ProgrammingError
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from lxml import etree

# necessary to allow import of local modules
cwd = os.getcwd()
sys.path.append(cwd)

from connection import Connection
db = Connection("postgresql+psycopg://ying.tan@localhost/osm_db")

In [ ]:
sql = """
with transformed_baskets as (
    select b.id as basket_id, st_transform(b.geom, 2263) as basket_geom, 
    (select nct.gid from nyc_census_tracts nct where st_contains(nct.geom, st_transform(b.geom, 2263))) as gid 
    from baskets b

)
select t.gid, t.ntaname, count(basket_id) as num_baskets, ST_NumGeometries(t.geom), t.geom from nyc_census_tracts t
left join transformed_baskets b on t.gid = b.gid
group by t.gid order by t.gid asc;
"""


print("spot check that the number of baskets and polygons per census tract look correct.")
nct = gpd.GeoDataFrame.from_postgis(sql, db.engine)
nct.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
nct.plot('st_numgeometries', ax=ax, legend=True)
plt.title("number of gemoetries in each census tract")

In [ ]:
# Get a map of all the ways in nyc, plot them out.

# ways = gpd.GeoDataFrame.from_postgis("select e.id, st_transform(e.way, 4326) as geom from edges e", db.engine)
# ways = gpd.GeoDataFrame.from_postgis("select w.way_id, st_transform(w.geom, 4326) as geom from ways w", db.engine)
# nyct = gpd.read_file("../data/nyct2020_24c/nyct2020.shp")
# nyct.geometry = nyct.geometry.to_crs("EPSG:4326")


ways_sql = """
with ways4326 as (
	select w.way_id, st_transform(w.geom, 4326) as geom from ways w 
)
, geojson_ways as (
	select 
		st_asgeojson(w.*, geom_column => 'geom', id_column => 'way_id')::json as geojson,
		(row_number() over ())/10000 as row
	from ways4326 w
)

-- This agg json step will speed up the visualization in the map, because we are not creating many
-- individual linestrings
SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(w.geojson)
    ) as geojson
from geojson_ways w 
group by row

"""
ways = db.execute(ways_sql)
print("found ways")
problematic_baskets = db.execute("""
with baskets4326 as (
	select v.id, st_transform(v.geom, 4326) as geom from vertices v
    where v.id is not null
    and v.id in (20110063,20110066,20110261,20120001,20120003,20120016,20120019,20120020,20120021,20120022,20120028,20120030,20120031,20120032,20120033,20120036,20120037,20120042,20120043,20120044,20120045,20120052,20120054,20120056,20120057,20120059,20120063,20120076,20120084,20120086,20120102,20120105,20120106,20120107,20120111,20120112,20120114,20120117,20120118,20120135,20120136,20120137,20120150,20120152,20120154,20210003,20210004,20210006,20210012,20210013,20210014,20210015,20210016,20210017,20210018,20210021,20210022,20210023,20210036,20210047,20210049,20210051,20210053,20210055,20210060,20210062,20210064,20210066,20210068,20210070,20210072,20210074,20210078,20210093,20210095,20210096,20210098,20210107,20210111,20210112,20210120,20210121,20210146,20210150,20210151,20210164,20210166,20210167,20210168,20210169,20210170,20210171,20310002,20310004,20310021,20310036,20310043,20310045,20310051,20310061,20310076,20310077,20310124,20310125,20310130,20310135,20310137,20310140,20310141,20310142,20310151,20310152,20310197,20310198,20310211,20310212,20410062,20410089,20410092,20410104,20420006,20420008,20420012,20420014,20420017,20420028,20420029,20420031,20420032,20420033,20420034,20420040,20420043,20420044,20420045,20420049,20420050,20420051,20420052,20420054,20420055,20420056,20420057,20420058,20420064,20420065,20420073,20420074,20420078,20420079,20420080,20420082,20420085,20420086,20420087,20420088,20420091,20420092,20430002,20430003,20430004,20430014,20430055,20430057,20430064,20430065,20430067,20430068,20430079,20430080,20430102,20430108,20430109,20430110,20430111,20430112,20430114,20430122,20430127,20510006,20510010,20510014,20510015,20510016,20510018,20510021,20510029,20510034,20510041,20510043,20510045,20510046,20510059,20510060,20510062,20510063,20510066,20510070,20510071,20510072,20510076,20510079,20520006,20520007,20520009,20520011,20520016,20520038,20520042,20520043,20520044,20520045,20520057,20520058,20520059,20520060,20520061,20520062,20520063,20520064,20520065,20520066,20520070,20520111,20520114,20620082,20620083,20810041,20810053,20810054,20810055,20820017,20820020,20820022,20820023,20820024,20820025,20820026,20820057,20820060,20820061,20820062,20820063,20820064,20820065,20820069,20820080,20820097,20820102,20820114,20820115,20820117,20820118,20820124,20820143,20820144,20820148,20820150,20820152,20820153,20820156,20820157,20820159,20820160,20830013,20830015,20830029,20830032,20830033,20830035,20830036,20830041,20830042,20830044,20830050,20830052,20830064,20830073,20830098,20830099,20830100,20830101,20830102,20830103,20830104,20830105,20830106,21010023,21010024,21010025,21010028,21010029,21010039,21010040,21010069,21010106,21010107,21010123,21010143,21010144,21010160,21010161,21010165,21010166,21010169,21010170,21010171,21010181,21010187,21010188,21010219,21010220,21010221,21010228,21010234,21010236,21020001,21020002,21020003,21030003,21030004,21030005,21030006,21030007,21030008,21030009,21030010,21030012,21030013,21030018,21030019,21030020,21030021,21030022,21030029,21030031,21030033,21030039,21030045,21030046,21030047,21030048,21030049,21030050,21030051,21030052,21030053,21030054,21030055,21030056,21030057,21030058,21030059,21030060,21030064,21030065,21030066,21030068,21030069,21030074,21030078,21030080,21030082,21030086,21030087,21030088,21030091,21030092,21110001,21110003,21110004,21110008,21110009,21110010,21110011,21110012,21110013,21110014,21110015,21110016,21110017,21110018,21110019,21110020,21110021,21110024,21110026,21110027,21110028,21110031,21110032,21110033,21110036,21110037,21110038,21110039,21110040,21110041,21110042,21110043,21110044,21110048,21110062,21110064,21110066,21110070,21110072,21110074,21110075,21110087,21110088,21110089,21110090,21110113,21110121,21110122,21110123,21110124,21110127,21110128,21120001,21120017,21120020,21120034,21120036,21120037,21120038,21120039,21120040,21120041,21120042,21120043,21130002,21130003,21130016,21130017,21130018,21130019,21130020,21130021,21130034,21210003,21210004,21210005,21210006,21210009,21210014,21210015,21210016,21210017,21210018,21210020,21210021,21210022,21210023,21210024,21210027,21210029,21210030,21210031,21210032,21210033,21210034,21210043,21210053,21210054,21210063,21210064,21210070,21210084,21210090,21220019,21220020,21220021,21220022,21220023,21220025,21220026,21220027,21220028,21220029,21220030,21220031,21220032,21220033,21220034,21220035,21220043,21220049,21220065,21220066,21220067,21220068,21220069,21220070,21230002,21230004,21230008,21230009,21230010)
)
, geojson_baskets as (
	select 
		st_asgeojson(vs.*, geom_column => 'geom', id_column => 'id')::json as geojson,
		(row_number() over ())/10000 as row
	from baskets4326 vs
)

SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(w.geojson)
    ) as geojson
from geojson_baskets w 
group by row
""")
print("found baskets")

loc = [40.7237415632241,-73.97571566678945]

# maybe we can use st_asgeojson here for more speedup.

m = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=14)
for _, row in ways.iterrows():
    c = folium.GeoJson(data=row.geojson, style_function=lambda x: {"weight": 1})
    c.add_to(m)
for _, row in problematic_baskets.iterrows():
    c = folium.GeoJson(data=row.geojson, style_function=lambda x: {"color": 'red', "weight": 3})
    c.add_to(m)
m


In [ ]:


# plot number of baskets in each census tract
baskets = gpd.GeoDataFrame.from_postgis("select id, type, lat, lng, st_asbinary(st_transform(geom, 2263)) as geom from baskets", db.engine)
baskets.geom = baskets.geom.set_crs("EPSG:2263")
fig, ax = plt.subplots(figsize=(10, 10))
# ymin, ymax = 215000, 212000
# xmin, xmax = 985000, 987500
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
nct.plot('count', ax=ax, legend=True)
baskets.plot(ax=ax, markersize=0.5)

plt.title("number of litter baskets in each census tract")

In [ ]:

# Leaflet does not support 2263, so convert all geos to 4326
baskets4326 = baskets.geom.to_crs("EPSG:4326")
tracts4326 = gpd.GeoDataFrame({ 'gid': nct['gid'], 'geom': nct.geom.to_crs(crs=4326), 'count': nct['count'], 'cdtaname': nct['cdtaname'] })
tracts4326.set_geometry('geom', crs="EPSG:4326", inplace=True)
loc = list(baskets4326.get_coordinates().iloc[0])
loc = [40.7237415632241,-73.97571566678945]
# loc = [40.730610, -73.935242]
map = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=14)


for index, row in tracts4326.iterrows():
    geo_j = gpd.GeoSeries(row['geom']).to_json()
    c = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(f"gid: {row['gid']}\ncount : {row['count']}").add_to(c)
    c.add_to(map)

for index, row in baskets4326.get_coordinates().iterrows():
    c = folium.vector_layers.CircleMarker(
        [row['y'], row['x']], radius=0.5
    )
    map.add_child(c)

# This map can be used to spotcheck that calculations are done correctly
map


In [ ]:
# --select (ST_SquareGrid(0.001, ST_Transform(geom,4326))).* from nyc_census_tracts where gid = 1;
# Testing: Return a grid of centroids that cover the census tract with gid = 1
ellis_island_gid = 1

sql = f"""
select st_centroid((st_squaregrid(0.001, st_transform(t.geom,4326))).geom) as geom from nyc_census_tracts t where gid = {ellis_island_gid}
--select st_centroid((st_squaregrid(1000, st_transform(t.geom,2263))).geom) as geom from nyc_census_tracts t where gid = {ellis_island_gid}
"""
grids = gpd.GeoDataFrame.from_postgis(sql, db.engine)


In [ ]:
# and map it
center = nct.loc[nct.gid == ellis_island_gid].centroid.to_crs("EPSG:4326")[0]

loc = [center.y, center.x]
map = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=14, scrollWheelZoom=False)
for index, row in grids.geom.get_coordinates().iterrows():
    c = folium.vector_layers.CircleMarker(
        [row['y'], row['x']], radius=0.5
    )
    map.add_child(c)
    
print("preview grid generation that covers a particular census tract")
map

In [ ]:
# sanity check for finding the bounds of nyc
sql = """
select st_transform(st_setsrid(st_extent(x.geom), 2263), 4326) as geom from 
(select st_union(t.geom) as geom from nyc_census_tracts t) as x;
"""
nyc_extent = gpd.GeoDataFrame.from_postgis(sql, db.engine)
nyc_extent

map = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=9)
geo_j = nyc_extent.geom.to_json()
c = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})

c.add_to(map)


fig, ax = plt.subplots(figsize=(10, 10))
tracts4326.plot('count', ax=ax, legend=True)
nyc_extent.plot(ax=ax, alpha=0.3)
xmin, ymin, xmax, ymax = nyc_extent.total_bounds
print("bounds of nyc, which can be used to query overpass turbo", [ymin, xmin, ymax, xmax])

In [ ]:
from urllib.parse import quote_plus

query = """
[out:xml][timeout:25];

(
  way["highway"="footway"](40.49600922391659, -74.25715844001557, 40.91528149796315, -73.69921524007877);
  way["footway"="sidewalk"](40.49600922391659, -74.25715844001557, 40.91528149796315, -73.69921524007877);
);

// print results
(
  ._; 
  >>;
);

out;

"""

arg = quote_plus(query)
url = '"' + 'https://overpass-api.de/api/interpreter?data=' + arg + '"'

OVERPASS_FILENAME = "nyc_footpaths_only_recurse_down.osm"

# This command downloads the paths 
!curl -o {OVERPASS_FILENAME} {url}

In [ ]:
# The resulting file needs to be modified so all untagged nodes are given a tag - otherwise they won't be
# imported into the db below.

RESULTS_FILENAME = f"parsed.osm"

tree = etree.parse(OVERPASS_FILENAME)

for n in tree.iter("node"):
    # add a tag to untagged nodes so that they get parsed
    if len(n) == 0: 
        n.append(etree.Element("tag", k="foo", v="bar"))

tree.write(RESULTS_FILENAME)



                

In [ ]:

# For safety we drop the previous tables first
sql = """
-- these are used in the default import
drop table if exists osm_footpaths_point;
drop table if exists osm_footpaths_line;
drop table if exists osm_footpaths_polygon;
drop table if exists osm_footpaths_roads;

-- these are used in the lua style import
drop table if exists nodes;
drop table if exists ways;
"""

db.raw_execute(sql)

def run_default_import():
    # Now run the tool to import data into the postgres db:
    # Note that nyc_footpaths_only.style is a style file copied from:
    # /opt/homebrew/opt/osm2pgsql/share/osm2pgsql/default.style
    # It tells osm2pgsql how to import data into postgres
    !osm2pgsql -c -d osm_db -H localhost --prefix osm_footpaths -S nyc_footpaths_only.style "$(pwd)/{RESULTS_FILENAME}"

def run_lua_import():
    !osm2pgsql -c -d osm_db -H localhost --output=flex --style=simple_import.lua "$(pwd)/{RESULTS_FILENAME}"

run_lua_import()
    

In [ ]:
# get all the roads
sql = """ 
 select l.way as geom, highway, footway
 from osm_footpaths_line l
"""

ways = gpd.GeoDataFrame.from_postgis(sql, db.engine)

fig, ax = plt.subplots(figsize=(10, 10))
ways.plot('highway', ax=ax, legend=True)
ways.plot('footway', ax=ax, legend=True)
